In [1]:
import pandas as pd
import sys
import os
parent_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(parent_dir)

from utils import const

data_path = "../data/binance/BTCUSDT-1m-2024-all-with-indicators.csv"

df = pd.read_csv(data_path)
df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
df = df.set_index("open_time")

In [2]:
df["target"] = (df["close"].shift(-1) > df["close"]).astype(int)
df = df.dropna()

X = df[const.factor_cols]
y = df["target"]

X_train = X[X.index < const.split_time]
X_test = X[X.index >= const.split_time]
y_train = y[y.index < const.split_time]
y_test = y[y.index >= const.split_time]

In [8]:
!which python
!pip show torch

python not found
zsh:1: command not found: pip


In [4]:


import torch

import torch.nn as nn
import torch.optim as optim


factor = const.factor_cols[0]
median = X_train[factor].median()
env1_idx = X_train[factor] <= median
env2_idx = X_train[factor] > median

envs = [
    (torch.tensor(X_train[env1_idx].values, dtype=torch.float32), torch.tensor(y_train[env1_idx].values, dtype=torch.float32)),
    (torch.tensor(X_train[env2_idx].values, dtype=torch.float32), torch.tensor(y_train[env2_idx].values, dtype=torch.float32)),
]

class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)

def irm_penalty(logits, y):
    scale = torch.tensor(1.0, requires_grad=True)
    loss = nn.BCEWithLogitsLoss()(logits * scale, y)
    grad = torch.autograd.grad(loss, [scale], create_graph=True)[0]
    return torch.sum(grad**2)

input_dim = X_train.shape[1]
model = MLP(input_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_steps = 3000
lambda_irm = 1e2

for step in range(n_steps):
    total_loss = 0
    penalty = 0
    for X_e, y_e in envs:
        logits = model(X_e)
        loss = nn.BCEWithLogitsLoss()(logits, y_e)
        total_loss += loss
        penalty += irm_penalty(logits, y_e)
    loss = total_loss / len(envs) + lambda_irm * penalty / len(envs)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 500 == 0:
        print(f"Step {step}, Loss: {loss.item():.4f}, Penalty: {penalty.item():.4f}")

with torch.no_grad():
    importance = model.net[0].weight.abs().sum(dim=0).cpu().numpy()
    factor_importance = sorted(zip(const.factor_cols, importance), key=lambda x: -x[1])
    for name, score in factor_importance:
        print(f"{name}: {score:.4f}")

KeyboardInterrupt: 